In [1]:
import torch
from torch.utils.data import Dataset
import numpy as np
import brevitas.nn as qnn
from brevitas.quant import IntBias
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
# import brevitas

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

class DeepSetDataset(Dataset):
    def __init__(self, data_files, target_files):
        self.data = []
        self.targets = []
        for data_file, target_file in zip(data_files, target_files):
            data = np.load(data_file)
            targets = np.load(target_file)
            self.data.append(data)
            self.targets.append(targets)

    def __getitem__(self, index):
        fold_idx = index // len(self.data[0])
        item_idx = index % len(self.data[0])
        data = self.data[fold_idx][item_idx]
        target = self.targets[fold_idx][item_idx]
        return torch.from_numpy(data), torch.from_numpy(target)

    def __len__(self):
        return len(self.data) * len(self.data[0])

base_file_name = 'jet_images_c8_minpt2_ptetaphi_robust_fast'

def datamaker(base_file_name, batch_size=32, val_split=0.2):
    train_data_files = [f'./normalized_data3/x_train_{base_file_name}.npy']
    train_target_files = [f'./normalized_data3/y_train_{base_file_name}.npy']
    test_data_files = [f'./normalized_data3/x_test_{base_file_name}.npy']
    test_target_files = [f'./normalized_data3/y_test_{base_file_name}.npy']

    train_tempdataset = DeepSetDataset(train_data_files, train_target_files)

    train_indices, val_indices = train_test_split(
        range(len(train_tempdataset)),
        test_size=val_split,
        random_state=42
    )

    train_dataset = torch.utils.data.Subset(train_tempdataset, train_indices)
    val_dataset = torch.utils.data.Subset(train_tempdataset, val_indices)
    test_dataset = DeepSetDataset(test_data_files, test_target_files)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, val_loader, test_loader

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader


# class DeepSetDataset(Dataset):
#     def __init__(self, data_files, target_files):
#         self.data = []
#         self.targets = []
        
#         for data_file, target_file in zip(data_files, target_files):
#             data = np.load(data_file)
#             targets = np.load(target_file)
            
#             self.data.append(data)
#             self.targets.append(targets)
    
#     def __getitem__(self, index):
#         fold_idx = index // len(self.data[0])
#         item_idx = index % len(self.data[0])
        
#         data = self.data[fold_idx][item_idx]
#         target = self.targets[fold_idx][item_idx]
        
#         return torch.from_numpy(data), torch.from_numpy(target)
    
#     def __len__(self):
#         return len(self.data) * len(self.data[0])



# # Define the base file name and number of folds
# # base_file_name = 'jet_images_c8_minpt2_allfeats_robust_fast'
# base_file_name = 'jet_images_c8_minpt2_ptetaphi_robust_fast'

# def datamaker(base_file_name, batch_size=32):
#     train_data_files = [f'./normalized_data3/x_train_{base_file_name}.npy']
#     train_target_files = [f'./normalized_data3/y_train_{base_file_name}.npy']
#     test_data_files = [f'./normalized_data3/x_test_{base_file_name}.npy']
#     test_target_files = [f'./normalized_data3/y_test_{base_file_name}.npy']

#     train_tempdataset = DeepSetDataset(train_data_files, train_target_files)
#     test_dataset = DeepSetDataset(test_data_files, test_target_files)









# # Define the base file name and number of folds
# # base_file_name = 'jet_images_c8_minpt2_allfeats_robust_fast'
# base_file_name = 'jet_images_c8_minpt2_ptetaphi_robust_fast'

# # Split the training data into training and validation sets
# train_data, val_data, train_targets, val_targets = train_test_split(train_dataset.data, train_dataset.targets, test_size=0.2, random_state=42)

# # Create the training and validation datasets
# train_dataset = DeepSetDataset(train_data, train_targets)
# val_dataset = DeepSetDataset(val_data, val_targets)

# # Create the training and validation data loaders
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

train_loader, val_loader, test_loader = datamaker(base_file_name, batch_size=32)

def test_model(model, test_loader):
    model.eval()
    with torch.no_grad():
        test_correct = 0
        test_total = 0
        for test_data, test_targets in test_loader:
            test_data = test_data.to(device).float()
            test_targets = test_targets.to(device).float()
            
            outputs = model(test_data)
            _, predicted = torch.max(outputs, 1)
            true_labels = torch.argmax(test_targets, 1)  # Get the true class labels
            
            test_total += true_labels.size(0)
            test_correct += (predicted == true_labels).sum().item()
        
        test_accuracy = test_correct / test_total
        print(f"Test Accuracy: {test_accuracy:.4f}")



In [3]:
class DeepSetsInv(nn.Module):
    def __init__(self, input_size, nnodes_phi: int = 32, nnodes_rho: int = 16, activ: str = "relu"):
        super(DeepSetsInv, self).__init__()
        self.nclasses = 5
        self.phi = nn.Sequential(
            qnn.QuantLinear(input_size, nnodes_phi, bias=True, weight_bit_width=8),
            self.get_activation(activ),
            qnn.QuantLinear(nnodes_phi, nnodes_phi, bias=True, weight_bit_width=8),
            self.get_activation(activ),
            qnn.QuantLinear(nnodes_phi, nnodes_phi, bias=True, weight_bit_width=8),
            self.get_activation(activ),
        )
        self.rho = nn.Sequential(
            qnn.QuantLinear(nnodes_phi, nnodes_rho, bias=True, weight_bit_width=8),
            self.get_activation(activ),
            qnn.QuantLinear(nnodes_rho, self.nclasses, bias=True, weight_bit_width=8),
        )

    def get_activation(self, activ):
        if activ == "relu":
            return qnn.QuantReLU(bit_width=8)
        elif activ == "sigmoid":
            return qnn.QuantSigmoid(bit_width=8)
        elif activ == "tanh":
            return qnn.QuantTanh(bit_width=8)
        else:
            raise ValueError(f"Unsupported activation: {activ}")
    
    def forward(self, inputs):
        phi_output = self.phi(inputs)
        # print("phi_output dtype:", phi_output.dtype)
        sum_output = torch.mean(phi_output, dim=1)
        # print( "sum_output dtype:", sum_output.dtype)
        rho_output = self.rho(sum_output)
        # print( "rho_output dtype:", rho_output.dtype)
        return rho_output



In [4]:
# Create the model
input_size = 3  # Assuming each input feature vector has a size of 16
model = DeepSetsInv(input_size=input_size, nnodes_phi=32, nnodes_rho=16, activ="relu")
# model.qconfig = IntBias.IntQuant.UINT8


# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()


optimizer = torch.optim.Adam(model.parameters(), lr=0.0032)
lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 2
model.to(device)

# Training loop
best_val_accuracy = 0.0
patience= 7
for epoch in range(num_epochs):
    lossval = 0.0
    model.train() 
    for batch_data, batch_targets in train_loader:
        batch_data = batch_data.to(device).float()
        batch_targets = batch_targets.to(device).float()
        
        # Forward pass
        outputs = model(batch_data)
        
        loss = criterion(outputs, batch_targets)
        lossval += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()  
    with torch.no_grad():
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        for val_data, val_targets in val_loader:
            val_data = val_data.to(device).float()
            val_targets = val_targets.to(device).float()
            
            outputs = model(val_data)
            loss = criterion(outputs, val_targets)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            true_labels = torch.argmax(val_targets, 1)  # Get the true class labels
            
            val_total += true_labels.size(0)
            val_correct += (predicted == true_labels).sum().item()

        val_accuracy = val_correct / val_total
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter > patience:
                print(f"Early stopping: Validation accuracy has not increased in {patience} epochs")
                break

        # Update the learning rate based on validation accuracy
        lr_scheduler.step(val_accuracy)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {lossval/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")
test_model(model, test_loader)

/opt/miniconda3/envs/morph_deepsets/lib/python3.10/site-packages/torch/_tensor.py:1394: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1711403226260/work/c10/core/TensorImpl.h:1911.)
  return super().rename(names)


Epoch 1/2, Train Loss: 1.0810, Val Loss: 1.0299, Val Accuracy: 0.5991
Epoch 2/2, Train Loss: 1.0275, Val Loss: 1.0194, Val Accuracy: 0.6073
Test Accuracy: 0.6074


In [5]:
# model = DeepSetsInv(input_size=input_size, nnodes_phi=32, nnodes_rho=16, activ="relu")
# model.qconfig = brevitas.quant.IntQuant.UINT8